In [14]:
import math
import h5py
import pandas as pd
import numpy as np
import tensorflow as tf
import os, glob
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot
np.random.seed(1)

In [15]:
# def parse_data(input):
#     if input is None:
#         return None
#     splits = tf.string_split([input], delimiter='#')
#     user_id = splits.values[0]
#     label_str = splits.values[1]
#     label_value = tf.string_to_number(label_str, tf.float32)
#     label = tf.clip_by_value(label_value, 0.0, 1.0, name=None)
#     activity_indices = tf.string_split([splits.values[2]], delimiter=',')
#     dense_activity_indices = tf.sparse_to_dense(
#         sparse_indices=activity_indices.indices,
#         output_shape=activity_indices.dense_shape,
#         sparse_values=activity_indices.values,
#         default_value='0',
#         validate_indices = True,
#         name = None
#     )
#     activities = tf.string_to_number(dense_activity_indices, out_type=tf.int32)
#     return (user_id, label, activities)

In [16]:
# def read_data(data_path):
#     if data_path is None:
#         return None
#     file_pattern = os.path.join(data_path, 'part-*')
#     files = tf.gfile.Glob(file_pattern)
#     dataset = tf.data.TextLineDataset(files)
#     return dataset.map(parse_data).batch(50).shuffle(buffer_size=10000)

In [17]:
# data = read_data('datasets/rnn').make_one_shot_iterator().get_next()

# iter = read_data('datasets/rnn').make_one_shot_iterator()
# print(iter.get_next())
# user_id, label, activities = iter.get_next()

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     (user_id, label, activities) = sess.run([user_id, label, activities])
# print(user_id)
# print(label)
# print(activities)

In [18]:
# def read_data(file_path):
#     if file_path is None:
#         return None
#     data = pd.read_csv(file_path,  sep='#|,', header=None, engine='python')
#     msk = np.random.rand(len(data)) < 0.8
#     train_data = data[msk]
#     test_data = data[~msk]
#     user_ids_train = np.array([train_data.iloc[:, 0].values])
#     labels_train = np.array([train_data.iloc[:, 1].values])
#     features_train = train_data.iloc[:,2:].T.values
#     user_ids_test = np.array([test_data.iloc[:, 0].values])
#     labels_test = np.array([test_data.iloc[:, 1].values])
#     features_test = test_data.iloc[:,2:].T.values
#     return user_ids_train, features_train, labels_train, user_ids_test, features_test, labels_test

In [19]:
def read_data(data_path):
    if data_path is None:
        return None
    data_files = glob.glob(os.path.join(data_path, 'part-*'))
    data = pd.concat(map(lambda file: pd.read_csv(file, sep='#|,', header=None, engine='python'), data_files), axis = 0, ignore_index = True)
    train_data, validate_data, test_data = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])
    
    user_ids_train = np.array([train_data.iloc[:, 0].values])
    labels_train = np.array([train_data.iloc[:, 1].values])
    features_train = train_data.iloc[:,2:].T.values
      
    user_ids_validate = np.array([validate_data.iloc[:, 0].values])
    labels_validate = np.array([validate_data.iloc[:, 1].values])
    features_validate = validate_data.iloc[:,2:].T.values
    
    user_ids_test = np.array([test_data.iloc[:, 0].values])
    labels_test = np.array([test_data.iloc[:, 1].values])
    features_test = test_data.iloc[:,2:].T.values
    
    return user_ids_train, features_train, labels_train, user_ids_validate, features_validate, labels_validate, user_ids_test, features_test, labels_test

In [20]:
def create_place_holders(n_x, n_y):
    if n_x is None or n_y is None:
        return None
    X = tf.placeholder(tf.float32, [n_x, None], "X")
    y = tf.placeholder(tf.float32, [n_y, None], "y")
    return X, y

In [21]:
def initialize_parameters(nx, layers):
    if nx is None or layers is None or len(layers) == 0:
        return None
    parameters = {}
    for i, k in enumerate(layers):
        parameters['W' + str(i+1)] = tf.get_variable('W' + str(i+1), [layers[i], nx], dtype=tf.float32, initializer = tf.contrib.layers.xavier_initializer(seed=1))
        parameters['b' + str(i+1)] = tf.get_variable('b' + str(i+1), [layers[i], 1],  dtype=tf.float32, initializer = tf.zeros_initializer())
        nx = layers[i]
    return parameters
        

In [22]:
def forward_propogation(X, parameters):
    if X is None or parameters is None or len(parameters) == 0:
        return None
    A = X
    for i in range(int(len(parameters)/2)):
        if i == (len(parameters)/2) - 1:
            Z = tf.add(tf.matmul(parameters['W' + str(i+1)], A), parameters['b' + str(i+1)])
            return Z
        Z = tf.add(tf.matmul(parameters['W' + str(i+1)], A), parameters['b' + str(i+1)])
        A = tf.nn.relu(Z) 
    return Z

In [23]:
def compute_cost(Z, Y, beta=0.001):
    if Z is None or Y is None:
        return None
    logits = tf.transpose(Z)
    labels = tf.transpose(Y)    
    L2 = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'W' in v.name]) * beta
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels) + L2)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits, labels = labels))
    return cost

In [24]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed=0):
    m = X.shape[1]                 
    mini_batches = []
    
    np.random.seed(seed)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    num_complete_minibatches = math.floor(m/mini_batch_size)
    
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [25]:
def model(X_train, Y_train, X_validate, Y_validate, X_test, Y_test, layers, learning_rate = 0.0001, num_epochs = 1500, minibatch_size = 32, print_cost = True):
    if X_train is None or Y_train is None or X_validate is None or Y_validate is None or X_test is None or Y_test is None or layers is None or len(layers) == 0:
        return None
    ops.reset_default_graph()
    (n_x, m) = X_train.shape
    (n_y, m) = Y_train.shape
    tf.set_random_seed(1) 
    seed = 3 
    costs = []
    
    X, Y = create_place_holders(n_x, n_y)
    parameters = initialize_parameters(n_x, layers)
    Z = forward_propogation(X, parameters)
    cost = compute_cost(Z, Y)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    
    num_minibatches = int(m / minibatch_size)
    
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            epoch_cost = 0.0
            seed = seed+1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})                
                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
      # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print("Parameters have been trained!")

        # Calculate the correct predictions
        #correct_prediction = tf.equal(tf.argmax(Z), tf.argmax(Y))
        correct_prediction = tf.equal(tf.round(tf.sigmoid(Z)), Y)

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print("Validation Accuracy:", accuracy.eval({X: X_validate, Y: Y_validate}))
        print("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
    return parameters
    

In [26]:
# working model
# user_id_train, X_train, Y_train, user_id_validate, X_validate, Y_validate, user_id_test, X_test, Y_test = read_data('datasets/rnn/566_3_100_v2')
# layers =[X_train.shape[0], 12, 1]

# print(user_id_train.shape)
# print(X_train.shape)
# print(Y_train.shape)

# print(user_id_test.shape)
# print(X_test.shape)
# print(Y_test.shape)

#parameters = model(X_train, Y_train, X_validate, Y_validate, X_test, Y_test, layers)


# (1, 35989)
# (100, 35989)
# (1, 35989)
# (1, 8990)
# (100, 8990)
# (1, 8990)


(100, 35741)
(1, 35741)
[[ 54   5 123 ... 104 147   4]
 [ 20 263  99 ...  32 251   1]
 [132 175 141 ...  30 240  54]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
[[1 0 0 ... 0 0 1]]
